# UnitTest

### Python

[Библиотека](https://docs.python.org/3.3/library/unittest.html) входит в состав языка Python

Основной шаблон тестирования

----------------------------------------

```python
import unittest

class SimplisticTest(unittest.TestCase):

    def test(self):
        self.assertTrue(True)

if __name__ == '__main__':
    unittest.main()
```

![](img/st1.png)

----------------------------------------


Тесты направлены на проверку определенного результата. Имеют обратную связь по выполнению: OK, ERROR, FAIL

```python
import unittest

class OutcomesTest(unittest.TestCase):

    def test_pass(self):
        self.assertTrue(True)

    def test_fail(self):
        self.assertTrue(False)

    def test_error(self):
        raise RuntimeError('Test error!')

if __name__ == '__main__':
    unittest.main()
```

![](img/st2.png)


### Тесты с определенным сообщение о ошибке

```python
import unittest

class FailureMessageTest(unittest.TestCase):

    def test_fail(self):
        self.assertTrue(False, 'Разместим текст ошибки - Что-то пошло не так')

if __name__ == '__main__':
    unittest.main()
```

### Проверка условий (True - удачное завершение, False - не удачное)

```python

import unittest

class TruthTest(unittest.TestCase):

    def test_assert_true(self):
        self.assertTrue(True)

    def test_assert_false(self):
        self.assertFalse(False)

if __name__ == '__main__':
    unittest.main()
    
```

### Тестирование исключений и предупреждений

Можно сделать `TestCase` который будет обеспечивать проверку исключений:

* [`TestCase` class](http://docs.python.org/3.3/library/unittest.html#unittest.TestCase)
* [`assert*` methods](http://docs.python.org/3.3/library/unittest.html#assert-methods)

<table>
  <tr><th>Method</th></tr>
  <tr><td><code>assertRaises(exception)</code></td></tr>
  <tr><td><code>assertRaisesRegex(exception, regexp)</code></td></tr>
  <tr><td><code>assertWarns(warn, fun, *args, **kwds)</code></td></tr>
  <tr><td><code>assertWarnsRegex(warn, fun, *args, **kwds)</code></td></tr>
</table>


```python
import unittest

def raises_error(*args, **kwds):
    raise ValueError('Не подходящее значение: %s%s' % (args, kwds))

class ExceptionTest(unittest.TestCase):

    def test_trap_locally(self):
        try:
            raises_error('a', b='c')
        except ValueError:
            pass
        else:
            self.fail('Другая ошибка не ValueError')

    def test_assert_raises(self):
        self.assertRaises(ValueError, raises_error, 'a', b='c')

if __name__ == '__main__':
    unittest.main()
```

Учтите, что метод `assertRaises()` является более правильным 

```
$ python3 test_exception.py -v

test_assert_raises (__main__.ExceptionTest) ... ok
test_trap_locally (__main__.ExceptionTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK
```
                      

### Основные виды тестов

| Методы |
|:------:|
|assertTrue(x, msg=None)|
|assertFalse(x, msg=None)|
|assertIsNone(x, msg=None)|
|assertIsNotNone(x, msg=None)|
|assertEqual(a, b, msg=None)|
|assertNotEqual(a, b, msg=None)|
|assertIs(a, b, msg=None)|
|assertIsNot(a, b, msg=None)|
|assertIn(a, b, msg=None)|
|assertNotIn(a, b, msg=None)|
|assertIsInstance(a, b, msg=None)|
|assertNotIsInstance(a, b, msg=None)|

### Остальные тесты

| Методы |
|:------:|
|assertAlmostEqual(a, b, places=7, msg=None, delta=None)|
|assertNotAlmostEqual(a, b, places=7, msg=None, delta=None)|
|assertGreater(a, b, msg=None)|
|assertGreaterEqual(a, b, msg=None)|
|assertLess(a, b, msg=None)|
|assertLessEqual(a, b, msg=None)|
|assertRegex(text, regexp, msg=None)|
|assertNotRegex(text, regexp, msg=None)|
|assertCountEqual(a, b, msg=None)|
|assertMultiLineEqual(a, b, msg=None)|
|assertSequenceEqual(a, b, msg=None)|
|assertListEqual(a, b, msg=None)|
|assertTupleEqual(a, b, msg=None)|
|assertSetEqual(a, b, msg=None)|
|assertDictEqual(a, b, msg=None)|


-------------------------

Узнать больше о тестах в Python можно [здесь](https://pymotw.com/3/unittest/index.html)

# PySpark Test

В качестве библиотеке тестирования возьмем [pytest](https://docs.pytest.org/en/stable/)

### rdd

In [ ]:
# подсчет слов

def wc(lines):
    """ 
        считаем слова в rdd
    """

    counts = (
             lines.flatMap(lambda x: x.split())
                  .map(lambda x: (x, 1))
                  .reduceByKey(add)
             ) 
    
    results = {word: count for word, count in counts.collect()}
    
    return results

как сделать тест для этой функции? Надо учитывать, что это pyspark

In [ ]:
def quiet_py4j():
    """ 
        отключаем логирование для тестовых запусков
    """
    logger = logging.getLogger('py4j')
    logger.setLevel(logging.WARN)


@pytest.fixture(scope="session")
def spark_context(request):
    """ 
        Добавляем запуск Spark Contexta
        
        request: pytest.FixtureRequest объект
    """
    
    conf = (SparkConf().setAppName("test_wc"))
    
    sc = SparkContext(conf=conf)
    
    request.addfinalizer(lambda: sc.stop())

    quiet_py4j()
    
    return sc



pytestmark = pytest.mark.usefixtures("spark_context")

def test_wc(spark_context):
    """ 
        тестируем функцию посчета слов
    
        spark_context: Spark SparkContext с параметрами для теста
    """
    
    # создаем простой пример
    test_input = [
        ' hello spark ',
        ' hello again spark spark'
    ]
    
    # пример в rdd
    input_rdd = spark_context.parallelize(test_input, 1)
    
    # запускаем функцию подсчета
    results = wordcount.wc(input_rdd)
    
    expected_results = {'hello':2, 'spark':3, 'again':1}  
    
    # проверяем результат
    assert results == expected_results

### DataFrame

In [6]:
# функция обработки DF
def jsoner(df, target_name):
    """ 
        находит записи с определенным именем
        name=target_name в PySpark DataFrame 
        в колонке 'name' 
    """
    
    return df.filter(df.name == target_name).count()

In [ ]:
# тестирование

# создаем hive_context (или SparkSession)
@pytest.fixture(scope="session")
def hive_context(spark_context):
    """  
        создаем hive_context на основе spark_context
    
        spark_context: spark_context 
        
        return: HiveContext
    """
    return HiveContext(spark_context)

pytestmark = pytest.mark.usefixtures("spark_context", "hive_context")

def test_jsoner(spark_context, hive_context):
    """ 
        тестирование функции фильтрации данные в DF
        
        spark_context: SparkContext
        hive_context: HiveContext
    """
    
    test_input = [
        {'name': 'Vasilii'},
        {'name': 'Vasisualii'},
        {'name': 'Vasilisa'},
        {'name': 'Vasian'},
    ]
    
    # создаем data frame на основе тестовых данных
    input_rdd = spark_context.parallelize(test_input, 1)
    df = hive_context.jsonRDD(input_rdd)
    
    # запускаем функцию проверки
    results = jsoncount.jsoner(df, 'Vasisualii')
    
    # ожидаемое кол-во найденых параметров
    expected_results = 1
    
    assert results == expected_results

In [9]:
### Spark Streaming / DStreanm

In [10]:
from operator import add

def streaming_wc(lines):

    counts_stream = (
                    lines.flatMap(lambda x: x.split())
                         .map(lambda x: (x, 1))
                         .reduceByKey(add)
                     )
    
    return counts_stream

In [11]:
def make_dstream_helper(sc, ssc, test_input):
    """ 
        создаем DStream из test_inputt_input
    
        test_input: rdd 
    
        return: dstream
    """
    input_rdds = [sc.parallelize(d, 1) for d in test_input]
    input_stream = ssc.queueStream(input_rdds)
    return input_stream


def collect_helper(ssc, dstream, expected_length, block=True):
    """
        собирает RDD в список 
        (базируется на https://github.com/holdenk/spark-testing-base)
    
        :return: список rdd после collect
    """
    result = []

    def get_output(_, rdd):
        if rdd and len(result) < expected_length:
            r = rdd.collect()
            if r:
                result.append(r)

    dstream.foreachRDD(get_output)

    if not block:
        return result

    ssc.start()

    timeout = 2
    start_time = time.time()
    while len(result) < expected_length and time.time() - start_time < timeout:
        time.sleep(0.01)
    if len(result) < expected_length:
        print("timeout after", timeout)

    return result

In [ ]:
def test_streaming_wc(spark_context, streaming_context):
    """ 
        тестирование элементов в стриминге
        
        spark_context: SparkContext
        streaming_context: SparkStreamingContext
    """

    test_input = [
        [
            ' hello spark ',
            ' hello again spark spark'
        ],
        [
            ' hello there again spark spark'
        ],

    ]
    
    # создаем пример стриминговых данных (rdd / queueStream)
    input_rdds = [sc.parallelize(d, 1) for d in test_input]
    input_stream = ssc.queueStream(input_rdds)
    
    # запускаем функцию
     = wordcount_streaming.streaming_wc(input_stream)
    results = collect_helper(streaming_context, tally, 2)
    
    expected_results = [
        [('again', 1), ('hello', 2), ('spark', 3)],
        [('again', 1), ('there', 1), ('hello', 1), ('spark', 2)]
    ]

    assert results == expected_results